## Objetivo do notebook
#### O objetivo deste notebook é realizar a leitura de dados das origens do dataset disciplinas na camada bronze e gravar de forma versionada na camada silver


### Import Libs

In [1]:
!pip install minio

In [2]:
from pyspark.sql import SparkSession
from datetime import datetime
from minio import Minio
from pyspark.sql.functions import when, col, lit,isnan
from io import BytesIO

import os
import pandas as pd
import glob
import pyarrow.parquet as pq

### Definição de variáveis

In [3]:
# Sessão Spark
spark = SparkSession.builder.getOrCreate()

# Parametros de input e output das origens
camadaLeitura = 'bronze'
camadaEscrita = 'silver'
pasta = 'disciplinas'
temp_blobs = '/home/jovyan/notebooks/temporary_blobs/' # pasta temporária para armazenamento de objetos

# Conexão ao miniIO
minio_endpoint = 'minio:9000'
minio_access_key = 'minioaccesskey'
minio_secret_key = 'miniosecretkey'
minio_client = Minio(minio_endpoint, access_key=minio_access_key, secret_key=minio_secret_key, secure=False)

# Definição de variáveis para versionamento dos dados na camada bronze
timestamp = datetime.today().strftime('%Y%m%d')
minio_path = f'{pasta}/{pasta + timestamp}'

### Leitura da camada Bronze

In [4]:
dataframes = []

try:
    # Lista objetos no bucket
    objects = minio_client.list_objects(camadaLeitura, prefix=pasta, recursive=True)

    # Itera sobre os objetos na pasta
    for obj in objects:
        # Lê o conteúdo do objeto
        content = minio_client.get_object(camadaLeitura, obj.object_name).read()
        
        # Lê o conteúdo do objeto Parquet usando pyarrow e converte para DataFrame Pandas
        df_pandas = pq.read_table(BytesIO(content)).to_pandas()
        
        # Adiciona o DataFrame Pandas à lista de DataFrames
        dataframes.append(df_pandas)
        
        print(f"Objeto encontrado: {obj.object_name}")
except error.MinioException as e:
    print(f"Erro ao listar objetos: {e}")

# Concatena todos os DataFrames Pandas em um único DataFrame Pandas
consolidated_df_pandas = pd.concat(dataframes, ignore_index=True)

# Converte para DataFrame PySpark
df_disciplinas = spark.createDataFrame(consolidated_df_pandas)

Objeto encontrado: disciplinas / disciplinas20240114/part-00000-5f09ef2a-7027-431f-a363-b014b76eecd1-c000.snappy.parquet
Objeto encontrado: disciplinas / disciplinas20240114/part-00000-95df0857-4f1b-46a9-88f8-e813b9bf1dbd-c000.snappy.parquet
Objeto encontrado: disciplinas / disciplinas20240114/part-00000-b7d4935b-1992-4a7d-8cb9-1a1abb67f557-c000.snappy.parquet
Objeto encontrado: disciplinas / disciplinas20240114/part-00000-f5fd30b7-9d0a-47c2-9aa7-839e40025c20-c000.snappy.parquet
Objeto encontrado: disciplinas / disciplinas20240114/part-00001-5f09ef2a-7027-431f-a363-b014b76eecd1-c000.snappy.parquet
Objeto encontrado: disciplinas / disciplinas20240114/part-00001-95df0857-4f1b-46a9-88f8-e813b9bf1dbd-c000.snappy.parquet
Objeto encontrado: disciplinas / disciplinas20240114/part-00001-b7d4935b-1992-4a7d-8cb9-1a1abb67f557-c000.snappy.parquet
Objeto encontrado: disciplinas / disciplinas20240114/part-00001-f5fd30b7-9d0a-47c2-9aa7-839e40025c20-c000.snappy.parquet
Objeto encontrado: disciplinas /

### Tratamento dos dados

In [5]:
df_disciplinas = df_disciplinas.select(
    'id',
    'id_curso',
    'nome',
    'sigla',
    'dia_semana',
    'carga_horaria',
    # Remove campos sem nota e retonar 0
    when(isnan(col('avaliacoes')), lit(0)).otherwise(col('avaliacoes')).alias('avaliacoes'),
    'etl_date',
)

### Gravação do dataframe em um diretório temporário

In [6]:
df_disciplinas.write.parquet(temp_blobs, mode="overwrite")

### Gravação na camada Silver

In [7]:
# Recria bucket caso não exista
if not minio_client.bucket_exists(camadaEscrita):
    minio_client.make_bucket(camadaEscrita)

# Lista todos os arquivos Parquet no diretório temporário de blobs
arquivos_parquet = glob.glob(os.path.join(temp_blobs, '*.parquet'))

# Itera sobre a lista de arquivos Parquet e envia cada um para o MinIO
for arquivo_parquet in arquivos_parquet:
    try:
        # Envia o arquivo para a camada bronze
        nome_arquivo = os.path.basename(arquivo_parquet)
        minio_client.fput_object(camadaEscrita, os.path.join(minio_path, nome_arquivo), arquivo_parquet)
        print(f"Arquivo '{nome_arquivo}' enviado com sucesso para o MinIO em '{os.path.join(minio_path, nome_arquivo)}'.")
    except S3Error as e:
        print(f"Erro ao enviar o arquivo para o MinIO: {e}")

Arquivo 'part-00000-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet' enviado com sucesso para o MinIO em 'disciplinas/disciplinas20240114/part-00000-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet'.
Arquivo 'part-00001-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet' enviado com sucesso para o MinIO em 'disciplinas/disciplinas20240114/part-00001-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet'.
Arquivo 'part-00002-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet' enviado com sucesso para o MinIO em 'disciplinas/disciplinas20240114/part-00002-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet'.
Arquivo 'part-00003-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet' enviado com sucesso para o MinIO em 'disciplinas/disciplinas20240114/part-00003-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet'.
Arquivo 'part-00004-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet' enviado com sucesso para o MinIO em 'disciplinas/disci

### Remove dados do diretório temporário de blobs

In [8]:
# Liste todos os arquivos na pasta
arquivos_na_pasta = os.listdir(temp_blobs)

# Itere sobre os arquivos e os delete
for arquivo in arquivos_na_pasta:
    caminho_completo = os.path.join(temp_blobs, arquivo)
    try:
        if os.path.isfile(caminho_completo):
            os.remove(caminho_completo)
            print(f'{caminho_completo} deletado com sucesso.')
    except Exception as e:
        print(f'Erro ao deletar {caminho_completo}: {e}')

spark.stop()

/home/jovyan/notebooks/_temporary_blobs/.part-00000-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/.part-00001-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/.part-00002-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/.part-00003-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/.part-00004-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/.part-00005-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/.part-00006-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet.crc deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/.part-00007-c01

/home/jovyan/notebooks/_temporary_blobs/part-00002-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/part-00003-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/part-00004-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/part-00005-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/part-00006-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/part-00007-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/part-00008-c018727c-20e0-4b16-9712-d31f266b44e0-c000.snappy.parquet deletado com sucesso.
/home/jovyan/notebooks/_temporary_blobs/part-00009-c018727c-20e0-4b16-9712-d31f266b44e0-c0